In [1]:
import csv
import json
import os
import pickle
import random
import shutil
import typing
from concurrent.futures import ProcessPoolExecutor

import albumentations as A
import cv2
import lightning as L
import numpy as np
import scipy
import skimage
import skimage.filters
import skimage.io
import skimage.transform
import torch
from torch import nn
import torchvision
import tqdm
from albumentations.pytorch import ToTensorV2
from PIL import Image
from sklearn.neighbors import KNeighborsClassifier
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict
import torchmetrics

/Users/anymax/Documents/mipt/ШАД/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/anymax/Documents/mipt/ШАД/.venv/lib/python3.12/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
from rare_traffic_sign_solution import DatasetRTSD, TestData, CustomNetwork, train_model, train_simple_classifier, test_classifier, apply_classifier, generate_all_data

In [ ]:
def get_imgs_names(path_to_img):
    return os.listdir(path_to_img)

In [ ]:
len(get_imgs_names('./icons'))

205

In [ ]:
len(get_imgs_names('./smalltest'))

1309

In [ ]:
len(get_imgs_names('./background_images'))

1000

In [ ]:
ds_train = DatasetRTSD(
        root_folders=['./cropped-train'], 
        path_to_classes_json='./classes.json'
    )
len(ds_train)

79896

In [ ]:
!echo $PATH

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir simple_classifier/lightning_logs

In [ ]:
model = train_simple_classifier()

In [ ]:
# test / apply

In [3]:
generate_all_data(
    output_folder='./syntetic_icons',
    icons_path='./icons',
    background_path='./background_images',
    samples_per_class=900
)

0it [00:00, ?it/s]/Users/anymax/Documents/mipt/ШАД/.venv/lib/python3.12/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/Users/anymax/Documents/mipt/ШАД/.venv/lib/python3.12/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/Users/anymax/Documents/mipt/ШАД/.venv/lib/python3.12/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment 

In [ ]:
weights = torchvision.models.ResNet50_Weights.DEFAULT
model = torchvision.models.resnet50(weights=weights)
model.fc = nn.Sequential(OrderedDict([
        (f'lin1', nn.Linear(in_features=model.fc.in_features, out_features=1024)),
        (f'act', nn.ReLU()),
        (f'lin2', nn.Linear(in_features=1024, out_features=205)),
]))
list(model.children())[-2]

AdaptiveAvgPool2d(output_size=(1, 1))